# TextCNN

参数配置，训练数据生成，模型结构，训练模型。

In [78]:
import tensorflow as tf
import tensorflow.keras as K
import tensorflow.keras.layers as L
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer 

In [79]:
class TrainingConfig():
    epochs = 5
    batch_size= 128
    learningRate=0.001
    valRate=0.1
    loss=['sparse_categorical_crossentropy']
    metrics=['accuracy']
    optimizer='adam'
    
class WordEmbedding():
    sequenceLength=256
    wordCount=10000
    wordDim=10

class DataSourceConfig():
    source=["./dataset/imdb_train_data.npy",
           "./dataset/imdb_test_data.npy",
           "./dataset/imdb_train_labels.npy",
           "./dataset/imdb_test_labels.npy"]

In [80]:
class TextCNN():
    def __init__(self,config,wordEmbedding):
        
        self.model = None
        self.history = None
        self.config=config
        self.emb=wordEmbedding
        self.name="TextCNN"
      

    def design_model(self,hiden_size=64):
        input_layer=L.Input(shape=(None,),name='feature_input')
        x=L.Embedding(input_dim=self.emb.wordCount,output_dim=self.emb.wordDim,input_length=self.emb.sequenceLength)(input_layer)
        x=L.Conv1D(filters=100,kernel_size=5)(x)
        x=L.BatchNormalization()(x)
        x=L.Activation("relu")(x)
        x=L.GlobalMaxPool1D()(x)
        x=L.Dense(hiden_size,activation='relu',name="feature_output")(x)
        output_layer=L.Dense(2,activation='softmax')(x)
        model=K.models.Model(inputs=[input_layer],outputs=[output_layer],name=self.name)
        self.model=model
        model.summary()
        return model
    
    def compile_model(self):
        #assert()
        if not self.model:
            print("Call design_modelXX() to build the model first.")
            return
        self.model.compile(optimizer=self.config.optimizer,loss=self.config.loss,metrics=self.config.metrics)

    def train_model(self,x,y):
        if not self.model:
            print("Call design_modelXX() to build the model first.")
            return
        self.compile_model()
        self.history=self.model.fit(x=x,y=y,batch_size=self.config.batch_size,epochs=self.config.epochs,validation_split=self.config.valRate)
        #self.model.compile()
    
    def save_model(self):
        if not self.model:
            print("Call design_modelXX() to build the model first.")
        pass
    
    def eval_model(self):
        if not self.model:
            print("Call design_modelXX() to build the model first.")
        pass
    
    def view_train(self):
        pass
    #model=K.models.Model(inputs=[input_layer],outputs=[output_layer])
    #model.compile('adam',loss='categorical_crossentropy',metrics=metrics)
    #return model

In [81]:
train_config=TrainingConfig()
word_embedding=WordEmbedding()
model=TextCNN(train_config,word_embedding)

In [82]:
model.design_model()

Model: "TextCNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
feature_input (InputLayer)   [(None, None)]            0         
_________________________________________________________________
embedding_10 (Embedding)     (None, None, 10)          100000    
_________________________________________________________________
conv1d_10 (Conv1D)           (None, None, 100)         5100      
_________________________________________________________________
batch_normalization_10 (Batc (None, None, 100)         400       
_________________________________________________________________
activation_10 (Activation)   (None, None, 100)         0         
_________________________________________________________________
global_max_pooling1d_10 (Glo (None, 100)               0         
_________________________________________________________________
feature_output (Dense)       (None, 64)                6464

In [83]:
class Dataset():
    def __init__(self,config):
        self.config=config
    def load(self):
        L=[]
        for item in self.config.source:
            L.append(np.load(item,allow_pickle=True, fix_imports=True))
        return L

In [84]:
data=Dataset(DataSourceConfig)
train_data,test_data,train_labels,test_labels=data.load()

In [85]:
model.train_model(x=train_data,y=train_labels)

Train on 22500 samples, validate on 2500 samples
Epoch 1/5
22500/22500 [==============================] - 24s 1ms/sample - loss: 0.5287 - accuracy: 0.7146 - val_loss: 0.6316 - val_accuracy: 0.8264
Epoch 2/5
22500/22500 [==============================] - 22s 982us/sample - loss: 0.2643 - accuracy: 0.8920 - val_loss: 0.4926 - val_accuracy: 0.8492
Epoch 3/5
22500/22500 [==============================] - 22s 962us/sample - loss: 0.1378 - accuracy: 0.9544 - val_loss: 0.3160 - val_accuracy: 0.8844
Epoch 4/5
22500/22500 [==============================] - 22s 994us/sample - loss: 0.0601 - accuracy: 0.9854 - val_loss: 0.3099 - val_accuracy: 0.8804
Epoch 5/5
22500/22500 [==============================] - 23s 1ms/sample - loss: 0.0195 - accuracy: 0.9975 - val_loss: 0.3902 - val_accuracy: 0.8760
